# Exploration: training a model

We create a pytorch-based model for image classification from public FashionMNIST dataset.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np

# Get cpu or gpu for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

class ImageClassifier(nn.Module):
  def __init__(self):
      super().__init__()
      self.model = nn.Sequential(
          nn.Conv2d(1, 8, kernel_size=3),
          nn.ReLU(),
          nn.Conv2d(8, 16, kernel_size=3),
          nn.ReLU(),
          nn.Flatten(),
          nn.LazyLinear(10),  # 10 classes in total.
      )

  def forward(self, x):
      return self.model(x)


training_data = datasets.FashionMNIST(
  root="data",
  train=True,
  download=True,
  transform=ToTensor(),
)
train_dataloader = DataLoader(training_data, batch_size=128)

loss_fn = nn.CrossEntropyLoss()
model = ImageClassifier().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

model.train()
for batch, (X, y) in enumerate(train_dataloader):
  X = X.to(device)
  y = y.to(device)

  pred = model(X)
  loss = loss_fn(pred, y)
  
  # Backpropagation.
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  if batch % 100 == 0:
      loss_value = loss.item()
      current = batch
      step = batch // 100
      print(f"loss: {loss_value:2f}")

In [ ]:
sample_input = training_data[0][0][None, :].numpy()      
# Get model output - convert tensor to numpy
with torch.no_grad():
    output = model(torch.tensor(sample_input))
    sample_output = output.numpy()
    print(sample_output)